In [1]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
from keras import models
from IPython import display

from vpt.features.features import *
import vpt.utils.image_processing as ip
import vpt.settings as s

%load_ext autoreload
%autoreload 2

Using Theano backend.


In [2]:
def generate_hand_data(fs, hd, annotations, progress_bar, show=False):

    X_lh = []
    y_lh = []

    X_rh = []
    y_rh = []

    filenames = []

    hg = HandGenerator(fs, hd, annotations)
    
    hgen = hg.hand_generator(debug=False)
    init_show = False
    for lh, rh in hgen:
        if lh.label() != None and rh.label() != None:
            progress_bar.update()
            # display
            if show:
                if not init_show:
                    plt.figure(figsize=(12,4))
                    plt.subplot(121)
                    disp_lh = plt.imshow(lh.get_hand_img())
                    
                    plt.subplot(122)
                    disp_rh = plt.imshow(rh.get_hand_img())
                    
                    init_show = True
                else:
                    print(lh.get_fpath())
                    disp_lh.set_data(lh.get_hand_img())
                    disp_rh.set_data(rh.get_hand_img())
                    plt.draw()
                    
                    display.clear_output(wait=True)
                    display.display(plt.gcf())
                    print(lh.get_fpath())
                    plt.pause(0.001)
            
            filenames.append(lh.get_fpath())
            
            y_lh.append(lh.label())
            X_lh.append(lh.get_hand_img())
            
            y_rh.append(rh.label())
            X_rh.append(rh.get_hand_img())
            
        else:
            raise RuntimeWarning("Warning: No label found for hands")

    X_lh = np.array(X_lh)
    y_lh = np.array(y_lh)
    filenames = np.array(filenames)
    
    return X_lh, y_lh, X_rh, y_rh, filenames

In [3]:
class ProgressBar:
    
    def __init__(self, total, bars=10):
        self.total = total
        self.bars = bars
        self.inc = total/self.bars
        self.barstr = ""
        self.count = 0
            
    def update(self):
        self.count += 1
        if self.count%self.inc == 0:
            self.barstr += "="
            print(self.barstr)

In [4]:
# Initialize Data Loading Parameters
s.participant = "p6"
s.sensor = "realsense"
s.note = "all_data_12_07"

ftype = "bin"
folder = "data/posture/p6/"
annotation_file = "data/posture/all/annotations.txt"

# RDF Parameters
refreshHD = False
M = 3
radius = .3
n_samples = 500
segmentation_model_path = "data/rdf/trainedmodels/{}_M{}_rad{:0.1f}".format("all", M, radius)

# Posture Parameters
feature_type = "cae"

In [5]:
annotations = load_annotations(annotation_file, debug=False)
fs = FileStream(folder, ftype=ftype, annotations=annotations, ignore=True)
rdf_hs = load_hs_model(s.participant, M, radius, n_samples , refreshHD, segmentation_model_path)

hd = HandDetector(rdf_hs)

progress_bar = ProgressBar(len(fs.get_fpaths()))

# Files Loaded: 3505
Loading existing hand segmentation model: data/rdf/trainedmodels/all_M3_rad0.3
Shape: 3505


/home/davidjo/vpt2/vpt2_py3/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/davidjo/vpt2/vpt2_py3/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
X_lh_p4, y_lh_p4, X_rh_p4, y_rh_p4, filenames_p4 = generate_hand_data(fs, hd, annotations, progress_bar, show=False)

=
Error Generating Image: ('Invalid Data', 'data/posture/p6/p6b/000866.bin', (0,))
==
===
====


In [7]:
np.save("data/posture/p6/hand_data_M3_rad.3/X_lh.npy", X_lh_p4)
np.save("data/posture/p6/hand_data_M3_rad.3/y_lh.npy", y_lh_p4)
np.save("data/posture/p6/hand_data_M3_rad.3/X_rh.npy", X_rh_p4)
np.save("data/posture/p6/hand_data_M3_rad.3/y_rh.npy", y_rh_p4)
np.save("data/posture/p6/hand_data_M3_rad.3/filenames.npy", filenames_p4)